In [2]:
##### Program 3.1 Planner solution to the static equilibrium model

import numpy as np
from scipy.optimize import minimize 


class CommandOptimum:
    '''Class of the command equilibrium model'''
    
    def __init__(self, Kbar = 10,
                 Lbar = 20, α = 0.3,
                 β1 = .3, β2 = .6):
        self.K = Kbar
        self.L = Lbar
        self.α = α
        self.β1 = β1
        self.β2 = β2
        self.K1, self.K2, self.L1, self.L2 = self.get_optimum()
        self.X1, self.X2 = self.get_output()
        
    def utility(self,x):
        return -((x[1]**self.β1*x[0]**(1-self.β1))**self.α\
                 *((self.L-x[1])**self.β2*(self.K-x[0])**(1-self.β2))**(1-self.α))
    
    
    def get_optimum(self):
        res = minimize(self.utility,[5.0,5.0])
        K1 = res.x[0]
        L1 = res.x[1]
        K2 = self.K - K1
        L2 = self.L - L1
        return K1,K2,L1,L2
    
    def get_output(self):
        X1 = self.L1**self.β1 * self.K1**(1-self.β1)
        X2 = self.L2**self.β2 * self.K2**(1-self.β2)
        return X1, X2
    
    
    def __repr__(self):
        return 'Model parameters:\nKbar = {}, Lbar = {}, α = {}\nβ1 = {}, β2 = {}\
        \n\nOptimum of this model is given by:\nL1 = {:.2f}, L2= {:.2f}\nK1 = {:.2f}, K2 = {:.2f}\nX1 = {:.2f}, X2 = {:.2f}'\
    .format(self.K,self.L,self.α,self.β1,self.β2,self.L1,self.L2,self.K1,self.K2,self.X1,self.X2)


model1 = CommandOptimum()
print(model1,'\n')

# model2 = CommandOptimum(Kbar = 20,Lbar = 15,α = .2,β1 = .3,β2 = .5)
# print(model2)

Model parameters:
Kbar = 10, Lbar = 20, α = 0.3
β1 = 0.3, β2 = 0.6        

Optimum of this model is given by:
L1 = 3.53, L2= 16.47
K1 = 4.29, K2 = 5.71
X1 = 4.04, X2 = 10.78 



In [28]:
# 3.2

class MarketSolution:
    '''Decentralized market solution'''
    
    def __init__(self, Kbar = 10,
                 Lbar = 20, α1 = 0.3,
                 α2 = 0.4, Tbar = 30,
                 β1 = .3, β2 = .6, p1 = 1.0,
                 leisure = False):
        self.lei = leisure
        self.K, self.L, self.α = Kbar, Lbar, α1
        if self.lei: self.α1, self.α2, self.T = α1, α2, Tbar
        self.β1, self.β2 = β1, β2
        self.p1 = p1
        self.p2, self.w, self.r = self.get_price()
        self.X1, self.X2= self.get_output()
        if self.lei:
            self.L1,self.L2,self.K1,self.K2, self.l = self.get_decisions()
        else:
            self.L1,self.L2,self.K1,self.K2 = self.get_decisions()
        
    def __repr__(self):
        if self.lei:
            return 'Vary labor supply:\n\nModel parameters:\nKbar = {}, Lbar = {}, Tbar = {}, α1 = {}, α2 = {}, β1 = {}, β2 = {}\
            \n\nOptimum of this model:\
            \n\nPrices:\np1 = {:.2f}, p2= {:.2f}, w = {:.2f}, r = {:.2f}\
            \n\nLabor, Capital, and Leisure choices: \nL1 = {:.2f}, L2= {:.2f}, K1 = {:.2f}, K2 = {:.2f}, l = {:.2f}\
            \n\nIncome and Output:\nY = {:.2f}, X1 = {:.2f}, X2 = {:.2f}'\
            .format(self.K,self.L,self.T,self.α1,self.α2,self.β1,self.β2,self.p1,self.p2,self.w,self.r,self.L1,self.L2,\
                    self.K1,self.K2,self.l,self.Y,self.X1,self.X2)
        else:
            return 'Constant labor supply:\n\nModel parameters:\nKbar = {}, Lbar = {}, α = {}, β1 = {}, β2 = {}\
            \n\nOptimum of this model:\
            \n\nPrices:\np1 = {:.2f}, p2= {:.2f}, w = {:.2f}, r = {:.2f}\
            \n\nLabor and Capital choices: \nL1 = {:.2f}, L2= {:.2f}, K1 = {:.2f}, K2 = {:.2f}\
            \n\nIncome and Output:\nY = {:.2f}, X1 = {:.2f}, X2 = {:.2f}'\
            .format(self.K,self.L,self.α 
                    ,self.β1,self.β2,self.p1,self.p2,self.w,self.r,self.L1,self.L2,\
                    self.K1,self.K2,self.Y,self.X1,self.X2)

    def market_clear(self,x):
        p2 = x[0]
        w = x[1]
        r = x[2]
        sol = np.zeros(3)
        sol[0] = 1/self.p1-(self.β1/w)**self.β1*((1-self.β1)/r)**(1-self.β1) 
        sol[1] = 1/p2-(self.β2/w)**self.β2*((1-self.β2)/r)**(1-self.β2)
        if self.lei:
            self.Y = w*self.T + r*self.K
            sol[2] = self.β1*self.α*self.Y/w + self.β2*self.α2*self.Y/w + (1-self.α -self.α2)*self.Y/w - self.T
        else:
            self.Y = w*self.L + r*self.K
            sol[2] = self.β1*self.α*self.Y/w + self.β2*(1-self.α)*self.Y/w - self.L
        return np.linalg.norm(sol)
    
    def get_price(self):
        res = minimize(self.market_clear,[.5,.5,.5],bounds = [(1e-13, None) for i in range(3)])
        return res.x
    
    def get_decisions(self):
        L1 = self.β1/self.w*self.p1*self.X1
        L2 = self.β2/self.w*self.p2*self.X2
        K1 = (1-self.β1)/self.r*self.p1*self.X1
        K2 = (1-self.β2)/self.r*self.p2*self.X2
        if self.lei:
            l = (1-self.α-self.α2)*self.Y/self.w
            return L1,L2,K1,K2,l
        else:
            return L1,L2,K1,K2
    
    def get_output(self):
        X1 = self.α/self.p1*self.Y
        if self.lei:
            X2 = self.α2/self.p2*self.Y
        else:
            X2 = (1-self.α)/self.p2*self.Y
            
        return X1, X2


model3=MarketSolution(leisure = True)
print(model3)

Vary labor supply:

Model parameters:
Kbar = 10, Lbar = 20, Tbar = 30, α1 = 0.3, α2 = 0.4, β1 = 0.3, β2 = 0.6            

Optimum of this model:            

Prices:
p1 = 1.00, p2= 0.90, w = 0.37, r = 0.64            

Labor, Capital, and Leisure choices: 
L1 = 4.29, L2= 11.43, K1 = 5.68, K2 = 4.32, l = 14.29            

Income and Output:
Y = 17.39, X1 = 5.22, X2 = 7.75
